# Env

In [ ]:
import requests
import numpy as np

In [ ]:
url = "http://127.0.0.1:8800"

# Mobile Base Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos = get_mobile_joint_position()
RESULT = {
    "mobile_joint_position": pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
set_mobile_target_joint([1.500, -3.450, PI])
set_mobile_target_joint([1.800, -3.450, 0])
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

# Arm Joint Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos = get_arm_joint_position()
RESULT = {
    "arm_joint_position": pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
set_arm_target_joint([-0.011, -1.032, 0.051, -2.260, 0.063, 1.523, 0.675])
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

# End Effector Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos, ori = get_ee_position()
RESULT = {
    "ee_position": pos,
    "ee_orientation": ori
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
move_ee_delta((-0.1, -0.1, -0.0))
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response